In [1]:
# Import necessary modules
import os
import pandas as pd
from dotenv import load_dotenv
from modules.paragraph_generation import generate_paragraph, generate_advertising_paragraph
from modules.data_processing import load_and_filter_data, preprocess_columns_simple, enrich_advertiser_data_with_domain
from modules.linkedin_requests import initialize_client, generate_linkedin_request, generate_followup_message

# Load API key
load_dotenv()
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

In [2]:
# Load and preprocess datasets
people_path = './data/YC F24 Founder People.csv'
personal_data_path = './data/Sudarshan Sridharan People.csv'

relevant_columns = [
    'full_name', 'title', 'location', 'company', 'company_linkedin',
    'company_description', 'sub_title', 'summary', 'current_job_description',
    'person_industry', 'skills', 'education', 'work_history', 'company_domain'
]

people_data = load_and_filter_data(people_path, relevant_columns)
people_data = preprocess_columns_simple(people_data)

personal_data = load_and_filter_data(personal_data_path, relevant_columns)
personal_data = preprocess_columns_simple(personal_data)

# Enrich personal data with domain information
personal_data = enrich_advertiser_data_with_domain(personal_data)

# Generate paragraphs
people_data['descriptive_paragraph'] = people_data.apply(generate_paragraph, axis=1)
personal_data['advertising_paragraph'] = personal_data.apply(generate_advertising_paragraph, axis=1)


In [3]:
personal_data

,full_name,title,location,company,company_linkedin,company_description,sub_title,summary,current_job_description,person_industry,skills,education,work_history,company_domain,domain_info,advertising_paragraph
0,Sudarshan Sridharan,Founder,"San Francisco Bay, San Francisco Bay Area, Uni...",Pipeline,https://www.linkedin.com/company/withpipeline,"Get more leads, close more deals. Pipeline set...","CEO, Pipeline",NaN,Pipeline puts LinkedIn prospecting and outreac...,"Technology, Information and Internet",Investing,Clemson University,"Pipeline, Fion Technologies, SaveMAPS, Second ...",withpipeline.com,Sign Up → Sign Up → NEW Supercharge Your Sales...,Sudarshan Sridharan is the Founder at Pipeline...


In [4]:
# Load the system prompt from a .txt file
with open('system_prompt.txt', 'r') as file:
    system_prompt = file.read()

# Initialize Anthropic client
client = initialize_client(ANTHROPIC_API_KEY)

# Use the first advertiser's data for testing
advertising_paragraph = personal_data['advertising_paragraph'].iloc[0]
domain_info = personal_data['domain_info'].iloc[0]

# Process only the first 10 entries
max_entries = 10

results = []

print("Generated LinkedIn Requests and Follow-Up Messages:\n" + "=" * 50)

for idx, row in people_data.iterrows():
    if idx >= max_entries:
        break
    
    descriptive_paragraph = row['descriptive_paragraph']
    
    # Generate the connection request
    connect_note = generate_linkedin_request(client, advertising_paragraph, descriptive_paragraph, domain_info, system_prompt)
    
    # Generate the follow-up message
    follow_up_message = generate_followup_message(client, advertising_paragraph, descriptive_paragraph, domain_info, system_prompt)
    
    # Collect results
    results.append({
        'Name': row['full_name'],
        'Connect Note': connect_note,
        'Follow-Up Message': follow_up_message
    })

# Create DataFrame from results
output_df = pd.DataFrame(results)

# Display the DataFrame
output_df.style.set_properties(**{'text-align': 'left'})

# Save the DataFrame to a CSV for reference
output_df.to_csv('linkedin_messages.csv', index=False)

Generated LinkedIn Requests and Follow-Up Messages:


In [5]:
output_df

,Name,Connect Note,Follow-Up Message
0,Edward Aryee,Hey Edward! Your journey from Google to foundi...,Thank you for accepting my connection! I know ...
1,Victor Cheng,Hey Victor! Really cool to see another YC foun...,Here's a natural follow-up message:\n\nHi Vict...
2,Daryl Budiman,Hey Daryl! Love the work you're doing with And...,Hi Daryl! Thanks for connecting! \n\nI noticed...
3,Finn Mallery,"Hi Finn! As a fellow startup founder, I really...",Hey Finn! Thanks for connecting. I noticed you...
4,Arvind V.,Hey Arvind! Fellow YC founder here - building ...,Here's a natural follow-up message:\n\nHey Arv...
5,Daniel Edrisian,Hey Daniel! Really cool to see another founder...,Hey Daniel! Thanks for connecting. I wanted to...
6,Shardool Patel,Hey Shardool! Congrats on Ryvn's YC acceptance...,Hey Shardool - thanks for connecting! \n\nReal...
7,Philipp Baumanns,Hi Philipp! Really cool to see another founder...,Hi Philipp! Awesome to connect. Noticed you're...
8,Akhil Gupta,Hey Akhil! Your path from JHU to founding Fres...,Hey Akhil! Thanks for connecting. I noticed yo...
9,Chidi Williams,Hi Chidi! Love seeing fellow founders in the Y...,Here's a natural follow-up message:\n\nHi Chid...
